In [1]:
import sys
import os
import pandas as pd
from pyhive import hive

In [2]:
conn=hive.connect(host='apvrp31353',port=3063,auth='LDAP',username=os.environ['usr'],
                  password=os.environ['psswd'],configuration={'mapreduce.map.memory.mb':'9000',
                                                     'mapreduce.map.java.opts':'-Xmx7200m',
                                                     'hive.auto.convert.join':'false', 
                                                             'mapreduce.job.queuename':'araadh_q1.arapi_sq1'})

In [3]:
"""
conn=hive.connect(host='dbslp0309',port=10001,auth='LDAP',username=os.environ['usr'],
                  password=os.environ['psswd'],configuration={'mapreduce.map.memory.mb':'9000',
                                                     'mapreduce.map.java.opts':'-Xmx7200m',
                                                     'hive.auto.convert.join':'false', 
                                                             'mapreduce.job.queuename':'araadh_q1.arapi_sq1'})
"""

"\nconn=hive.connect(host='dbslp0309',port=10001,auth='LDAP',username=os.environ['usr'],\n                  password=os.environ['psswd'],configuration={'mapreduce.map.memory.mb':'9000',\n                                                     'mapreduce.map.java.opts':'-Xmx7200m',\n                                                     'hive.auto.convert.join':'false', \n                                                             'mapreduce.job.queuename':'araadh_q1.arapi_sq1'})\n"

In [4]:
dates = []

for i in range(3, 7):
    m = '0' + str(i)
    for v in range(1, 32):
        dates.append('2020' + m + str(v).zfill(2)) 

In [5]:
cols = ['clm_hc_diag_code1', 'clm_hc_diag_code2', 'clm_hc_diag_code3', 'clm_hc_diag_code4', 'clm_hc_diag_code5','clm_hc_diag_code6', 'clm_hc_diag_code7', 
        'clm_hc_diag_code8', 'clm_hc_diag_code9', 'clm_hc_diag_code10']

In [6]:
for d in dates:
    print(d)
    print('Pulling Data')
    sql = """select cl_fst_serv_dt, invn_ctl_nbr, clm_hc_diag_code1, clm_hc_diag_code2,
    clm_hc_diag_code3, clm_hc_diag_code4, clm_hc_diag_code5, clm_hc_diag_code6, clm_hc_diag_code7,
    clm_hc_diag_code8, clm_hc_diag_code9, clm_hc_diag_code10, cl_proc_cd, cl_place_of_srcv, clm_patient_zip,
    clm_patient_state, clm_bill_prov_state, clm_bill_prov_zip, clm_patient_dob, clm_total_charge, cl_line_item_chrg_amt
    from pi_prod.837p_clm_analytic_data where batch_run_dt = '%s' 
    and cl_fst_serv_dt >= %s and cl_fst_serv_dt <= %s""" % (d, dates[0], dates[-1])
    df = pd.read_sql(sql, conn)
    df = df.drop_duplicates(subset=['invn_ctl_nbr'])
    print('Classifying Data')
    df['cov_19'] = 'Non-COVID'
    df['cov_19'][df['cl_proc_cd'].isin(['U0001', 'U0002', '87635', 'G2023', 
                                        'G2024', 'G2063','86328', '86769', '87635', '86318'])] = 'Testing'
    for i in cols:
        df['cov_19'][(df[i]=='U071')] = 'Confirmed'
        df['cov_19'][(df[i].isin(['U072', 'B9729'])) & (df['cov_19']!='Confirmed')] = 'Probable'
        df['cov_19'][df[i].isin(['J1289', 'B342']) & (df['cov_19']!='Confirmed') & (df['cov_19']!='Probable')] = 'Possible'
        df['cov_19'][(df[i].isin(['Z20828', 'J22', 'B349', 'B9721', 'J1281'])) & 
                      (df['cl_place_of_srcv'] == '21') & (df['cov_19']!='Confirmed') & (df['cov_19']!='Probable')] = 'Possible'
    print(df['cov_19'].value_counts())
    print('Saving Data')
    df.to_csv('data/COVID_vs_NON_regional/%s.csv' % d)

20200506
Pulling Data
Classifying Data
Non-COVID    467411
Confirmed      4225
Possible        525
Testing         349
Probable        171
Name: cov_19, dtype: int64
Saving Data
20200507
Pulling Data
Classifying Data
Non-COVID    488981
Confirmed      4697
Possible        611
Testing         557
Probable        252
Name: cov_19, dtype: int64
Saving Data
20200508
Pulling Data
Classifying Data
Non-COVID    607950
Confirmed      5345
Testing         716
Possible        637
Probable        271
Name: cov_19, dtype: int64
Saving Data
20200509
Pulling Data
Classifying Data
Non-COVID    177472
Confirmed      1663
Possible        291
Testing         290
Probable         44
Name: cov_19, dtype: int64
Saving Data
20200510
Pulling Data
Classifying Data
Series([], Name: cov_19, dtype: int64)
Saving Data
20200511
Pulling Data
Classifying Data
Non-COVID    697600
Confirmed      6595
Testing         914
Possible        835
Probable        292
Name: cov_19, dtype: int64
Saving Data
20200512
Pulling Dat

In [7]:
for d in dates[5:]:
    print(d)
    print('Pulling Data')
    sql = """select cl_fst_serv_dt, invn_ctl_nbr, clm_hc_diag_code1, cl_proc_cd,clm_patient_zip,
    clm_patient_state, clm_bill_prov_state, clm_bill_prov_zip, clm_patient_dob, clm_total_charge, cl_line_item_chrg_amt
    from pi_prod.837i_clm_analytic_data where batch_run_dt = '%s' 
    and cl_fst_serv_dt >= %s and cl_fst_serv_dt <= %s""" % (d, dates[0], dates[-1])
    df = pd.read_sql(sql, conn)
    df = df.drop_duplicates(subset=['invn_ctl_nbr'])
    print('Classifying Data')
    df['cov_19'] = 'Non-COVID'
    df['cov_19'][df['cl_proc_cd'].isin(['U0001', 'U0002', '87635', 'G2023', 
                                        'G2024', 'G2063','86328', '86769', '87635', '86318'])] = 'Testing'

    df['cov_19'][(df['clm_hc_diag_code1'] =='U071')] = 'Confirmed'
    df['cov_19'][(df['clm_hc_diag_code1'].isin(['U072', 'B9729'])) & (df['cov_19']!='Confirmed')] = 'Probable'
    df['cov_19'][df['clm_hc_diag_code1'].isin(['J1289', 'B342']) & (df['cov_19']!='Confirmed') & (df['cov_19']!='Probable')] = 'Possible'
    df['cov_19'][(df['cl_proc_cd'].isin(['Z20828', 'J22', 'B349', 'B9721', 'J1281'])) & (df['cov_19']!='Confirmed') & (df['cov_19']!='Probable')] = 'Possible'
    print(df['cov_19'].value_counts())
    print('Saving Data')
    df.to_csv('data/COVID_vs_NON_regional/%s_i.csv' % d)
    

20200506
Pulling Data
Classifying Data
Non-COVID    30144
Testing        429
Confirmed      150
Possible         5
Probable         1
Name: cov_19, dtype: int64
Saving Data
20200507
Pulling Data
Classifying Data
Non-COVID    31064
Testing        532
Confirmed      188
Possible         6
Probable         2
Name: cov_19, dtype: int64
Saving Data
20200508
Pulling Data
Classifying Data
Non-COVID    52533
Testing        742
Confirmed      207
Possible        10
Probable         2
Name: cov_19, dtype: int64
Saving Data
20200509
Pulling Data
Classifying Data
Non-COVID    13649
Testing        175
Confirmed       81
Possible         1
Name: cov_19, dtype: int64
Saving Data
20200510
Pulling Data
Classifying Data
Series([], Name: cov_19, dtype: int64)
Saving Data
20200511
Pulling Data
Classifying Data
Non-COVID    124916
Testing        1836
Confirmed       459
Possible          4
Probable          2
Name: cov_19, dtype: int64
Saving Data
20200512
Pulling Data
Classifying Data
Non-COVID    94481
T

In [ ]:
dates = []

for i in range(2, 6):
    m = '0' + str(i)
    for v in range(1, 32):
        dates.append('2019' + m + str(v).zfill(2))

In [ ]:
for d in dates:
    print(d)
    print('Pulling Data')
    sql = """select cl_fst_serv_dt, invn_ctl_nbr, clm_hc_diag_code1, clm_hc_diag_code2,
    clm_hc_diag_code3, clm_hc_diag_code4, clm_hc_diag_code5, clm_hc_diag_code6, clm_hc_diag_code7,
    clm_hc_diag_code8, clm_hc_diag_code9, clm_hc_diag_code10, cl_proc_cd, cl_place_of_srcv, clm_patient_zip,
    clm_patient_state, clm_bill_prov_state, clm_bill_prov_zip, clm_patient_dob, clm_total_charge, cl_line_item_chrg_amt
    from pi_prod.837p_clm_analytic_data where batch_run_dt = '%s' 
    and cl_fst_serv_dt >= %s and cl_fst_serv_dt <= %s""" % (d, dates[0], dates[-1])
    df = pd.read_sql(sql, conn)
    df = df.drop_duplicates(subset=['invn_ctl_nbr'])
    print('Classifying Data')
    df['cov_19'] = 'Historic'
    print(len(df))
    print('Saving Data')
    df.to_csv('data/COVID_vs_NON_regional/%s.csv' % d)

In [ ]:
for d in dates:
    print(d)
    print('Pulling Data')
    sql = """select cl_fst_serv_dt, invn_ctl_nbr, clm_hc_diag_code1, cl_proc_cd,clm_patient_zip,
    clm_patient_state, clm_bill_prov_state, clm_bill_prov_zip, clm_patient_dob, clm_total_charge, cl_line_item_chrg_amt
    from pi_prod.837i_clm_analytic_data where batch_run_dt = '%s' 
    and cl_fst_serv_dt >= %s and cl_fst_serv_dt <= %s""" % (d, dates[0], dates[-1])
    df = pd.read_sql(sql, conn)
    df = df.drop_duplicates(subset=['invn_ctl_nbr'])
    print('Classifying Data')
    df['cov_19'] = 'Historic'
    print('Saving Data')
    df.to_csv('data/COVID_vs_NON_regional/%s_i.csv' % d)

In [ ]:
sql = """select count(*) from pi_prod.837i_clm_analytic_data
where batch_run_dt = '20200401'"""

In [ ]:
df = pd.read_sql(sql, conn)

In [ ]:
df.head()

In [ ]:
sql_i = """select * from pi_usrs.covid_837i_claims_v2"""
sql_p = """select * from pi_usrs.covid_837p_claims"""

In [ ]:
df_i = pd.read_sql(sql_i, conn)
df_p = pd.read_sql(sql_p, conn)

In [ ]:
df_i_cols = []
for i in df_i.columns:
    df_i_cols.append(i.split('.')[1])
df_i.columns = df_i_cols

In [ ]:
df_p_cols = []
for i in df_p.columns:
    df_p_cols.append(i.split('.')[1])
df_p.columns = df_p_cols

In [ ]:
df_p.columns

In [ ]:
df = df_p

In [ ]:
df_i['claim_type'] = 'facility'
df_p['claim_type'] = 'provider'

In [ ]:
df = pd.concat([df_i, df_p])

In [ ]:
df['claim_type'].value_counts()

In [ ]:
df['Testing'] = 0

In [ ]:
df['Testing'][df['cl_proc_cd'].isin(['U0001', 'U0002', '87635', 'G2023', 
                                        'G2024', 'G2063','86328', '86769', '87635', '86318'])] = 1

In [ ]:
df['Testing'].sum()

In [ ]:
df.groupby('clm_bill_prov_state')['Testing'].sum()

In [ ]:
out = df.groupby(['cl_fst_serv_dt', 'clm_patient_state'])['Testing'].sum().reset_index()

In [ ]:
out.to_csv('test.csv', index=False)

In [ ]:
def ages(x):
    if x <= 3:
        return '0-3'
    elif x <= 18:
        return '4-18'
    elif x <= 30:
        return '19-30'
    elif x <= 50:
        return '31-50'
    elif x <= 64:
        return '51-64'
    elif x <= 85:
        return '65-85'
    else:
        return '86+'

In [ ]:
df['age'] = df['patinet_age_cl'].apply(lambda x: ages(x))

In [ ]:
df_out = df.groupby(['cl_fst_serv_dt', 'age'])['invn_ctl_nbr'].count().reset_index()

In [ ]:
df_out.head()

In [ ]:
df.to_csv('837.csv', index=False)